In [9]:
import pandas as pd
import json
import geopandas as gpd
import shapely
from matplotlib import pyplot as plt

In [10]:
centroids = gpd.read_file('centroids.geojson')

In [11]:
traslados_day0 = pd.read_csv('Day0_Full_Filtered.csv')

In [12]:
centromty_polygon = shapely.Polygon([
    (-100.351,25.705),
    (-100.353,25.671),
    (-100.307,25.662),
    (-100.278,25.673),
    (-100.268,25.683),
    (-100.268,25.703)
])

# create geodataframe
centromty_gdf = gpd.GeoDataFrame(geometry=[centromty_polygon])
centromty_gdp = gpd.GeoSeries([centromty_polygon]).__geo_interface__

In [13]:
# Extract the H3 index of the centroid
traslados_day0_keyonly = traslados_day0[['h3_origin','h3_destination','hour_origin','dsemana_origin','count']]

In [14]:
# Convert centroids geodataframe to dataframe
centroids_df = pd.DataFrame(centroids)

reemplazo_traslados = traslados_day0_keyonly.copy()

# Replace the h3_origin column in data_traslados_zmm with the corresponding geometry from centroids
reemplazo_traslados['h3_origin'] = reemplazo_traslados['h3_origin'].map(centroids_df.set_index('h3_polyfill')['geometry'])

# Replace the h3_destination column in data_traslados_zmm with the corresponding geometry from centroids
reemplazo_traslados['h3_destination'] = reemplazo_traslados['h3_destination'].map(centroids_df.set_index('h3_polyfill')['geometry'])

# Display the data
reemplazo_traslados.head()

,h3_origin,h3_destination,hour_origin,dsemana_origin,count
0,POINT (-100.27967 25.68893),POINT (-100.28269 25.69097),7,0,1
1,POINT (-100.27967 25.68893),POINT (-100.28269 25.69097),10,0,2
2,POINT (-100.27967 25.68893),POINT (-100.28269 25.69097),19,0,2
3,POINT (-100.27967 25.68893),POINT (-100.28269 25.69097),20,0,1
4,POINT (-100.27967 25.68893),POINT (-100.27622 25.69028),2,0,1


In [15]:
reemplazo_traslados.to_csv('Day0_Full_Filtered_Centroids.csv', index=False)

In [16]:
# Create line strings from the origin and destination points
reemplazo_traslados['h3_line'] = reemplazo_traslados.apply(lambda x: shapely.geometry.LineString([x['h3_origin'], x['h3_destination']]), axis=1)

In [ ]:
# Divide data into 3 dataframes: one for origin, one for destination and one for the lines
origen = reemplazo_traslados[['h3_origin','hour_origin','dsemana_origin','count']].copy()
origen['geometry'] = origen['h3_origin']
origen.drop(columns=['h3_origin'], inplace=True)
origen = gpd.GeoDataFrame(origen)

destino = reemplazo_traslados[['h3_destination','hour_origin','dsemana_origin','count']].copy()
destino['geometry'] = destino['h3_destination']
destino.drop(columns=['h3_destination'], inplace=True)
destino = gpd.GeoDataFrame(destino)

lineas = reemplazo_traslados[['h3_line','hour_origin','dsemana_origin','count']].copy()
lineas['geometry'] = lineas['h3_line']
lineas.drop(columns=['h3_line'], inplace=True)
lineas = gpd.GeoDataFrame(lineas)

In [ ]:
# Expprt to geojson
origen.to_file('Day0_FiltCent_origin.geojson', driver='GeoJSON')
destino.to_file('Day0_FiltCent_destination.geojson', driver='GeoJSON')